**Carga de librería**

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import unicodedata
import re
import requests
import time
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [37]:
import matplotlib
import plotly
import sklearn

print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("matplotlib:", matplotlib.__version__)
print("seaborn:", sns.__version__)
print("plotly:", plotly.__version__)
print("requests:", requests.__version__)
print("scikit-learn:", sklearn.__version__)

pandas: 2.2.2
numpy: 2.0.2
matplotlib: 3.10.0
seaborn: 0.13.2
plotly: 5.24.1
requests: 2.32.4
scikit-learn: 1.6.1


**Carga de datos desde la API de datos abierto**

In [25]:

def restaurar_nombres_columnas(DataFrame):
  """ Función que restaura los valores de las columnas un vez el proceso de
      conversión del archivo JSON a Dataframe pandas finaliza. La función
      realiza los siguientes procesos
      - remplaza '_' por un espacio simple ' '
      - remplaza ' an lisis' por ' análisis'
      - remplza 'acidez kcl' por 'acidez intercambiable'
      - Convierte los nombres de la columnas a tipo título
        (primera letra en mayúsculas)
      Entrada: tipo Dataframe.pandas. Dataframe a restaurar
      Salida: tipo Dataframe.pandas. Mismo Dataframe restaurado
  """
  DataFrame = DataFrame.copy()
  DataFrame.columns = (
      DataFrame.columns
      .str.replace('_', ' ')
      .str.replace(' an lisis', ' análisis')
      .str.replace('acidez kcl', 'acidez intercambiable')
      .str.title()                  # poner en formato título
  )
  return DataFrame


# Función principal
BASE = 'https://www.datos.gov.co/resource/ch4u-f3i5.json'
LIMIT = 50000
offset = 0

all_rows = []

while True:
    url = f'{BASE}?$limit={LIMIT}&$offset={offset}'
    print('Descargando offset', offset)

    r = requests.get(url)
    data = r.json()

    if len(data) == 0:
        print('No hay más datos.')
        break

    all_rows.extend(data)
    offset += LIMIT
    time.sleep(0.3)

Data = pd.DataFrame(all_rows)
print('Total filas:', len(Data))
Data = restaurar_nombres_columnas(Data)

Descargando offset 0
Descargando offset 50000
Descargando offset 100000
No hay más datos.
Total filas: 92738


**Normalización de las columnas y acondicionamiento y limpieza de propiedades químicas del suelo**

In [26]:
def normalize_column_name(DataFrame):
  """Normaliza los nombre de las columnas del DataFrame:
  - lowercase
  - elimina tildes y caracteres no ASCII
  - reemplaza secuencias no alfanuméricas por " "
  - colapsa varios guiones bajos
  - quita guiones bajos al inicio/fin

  Entrada: Tipo DataFrame, Dataframe que contiene la columnas a tratar
  Salida: Base de datos con nombres de las columnas normalizadas
  """
  for j in DataFrame.columns:
    col = str(j).lower()
    col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('ascii')
    col = re.sub(r'[^a-z0-9]+', ' ', col)
    col = col.strip('_')
    col = re.sub(r'_+', '_', col)
    DataFrame.rename(columns={j: col}, inplace=True)
  return DataFrame


def clean_numeric_series(DataFrame, Columnas):
  """
    Función de preprocesamiento y acondicionamiento del dataset en las atributos
    definidos en Columnas. La función realiza las siguientes operaciones:
    - reemplaza comas por puntos (compatibilidad con python, punto flotante)
    - elimina espacios al final y al inicio
    - elimina punto consecutivos
    - elimina caracteres especiales en los datos (ej: *,/, <, etc)
    - remplaza valores vacios, - o notación extraña <1.5 por nan (no identificados)

    Notas:
    - Esta función no retorna el dataframe a float, la columna permanecen en objeto string
    - Trata las situaciones <1.36 como datos no identificados nan
    - Sólo se considera columnas relacionadas con las variables químicas del suelo
    Entradas:
      Dataframe: Tipo Dataframe.pandas. Dataframe que contiene la información acondicionar
      Columnas: Tipo lista o tupla. Condensa la columna a procesar )
    Salida:
      Dataframe: tipo Dataframe.pandas. Se retorna el mismo Dataframe de entrada pero con los
                 valores de las columnas procesadas
  """
  for j in Columnas:
    s = DataFrame[j]
    s = s.astype(str).str.strip()
    lt_mask = s.str.match(r'^\s*<\s*\d', na=False)
    # reemplazar comas por puntos
    s = s.str.replace(',', '.', regex=False)
    # colapsar múltiples puntos consecutivos a uno solo (ej: '4..01' -> '4.01')
    s = s.str.replace(r'\.+', '.', regex=True)
    # eliminar todo lo que no sea dígito, punto o signo negativo
    s = s.str.replace(r'[^0-9\.\-]', '', regex=True)
    # quitar casos que sean solo '.' o '-' o vacío
    s = s.replace({'': np.nan, '.': np.nan, '-': np.nan})
    if lt_mask.any():
        s.loc[lt_mask] = np.nan
    DataFrame[j] = s
  return DataFrame


# Función principal
# Se define las columnas acondicionar
Columnas_Filtro = ['ph agua suelo',
       'materia organica', 'fosforo bray ii', 'azufre fosfato monocalcico',
       'acidez intercambiable', 'aluminio intercambiable',
       'calcio intercambiable', 'magnesio intercambiable',
       'potasio intercambiable', 'sodio intercambiable',
       'capacidad de intercambio cationico', 'conductividad electrica',
       'hierro disponible olsen', 'cobre disponible',
       'manganeso disponible olsen', 'zinc disponible olsen',
       'boro disponible', 'hierro disponible doble acido',
       'cobre disponible doble acido', 'manganeso disponible doble acido',
       'zinc disponible doble acido']

PDatos = normalize_column_name(Data.copy())
PDatos = clean_numeric_series(PDatos, Columnas_Filtro)
# Las columnas procesadas se pasan de formato object string a float
PDatos[Columnas_Filtro] = PDatos[Columnas_Filtro].astype(float)
# Se adiciona una nueva columna año de medicción
PDatos['fecha de analisis'] = pd.to_datetime(PDatos['fecha de analisis'], dayfirst=True)
# Nueva columna solo con el año
PDatos['año'] = PDatos['fecha de analisis'].dt.year

**Agrupamiento por municipios en el valle del rio cauca (Cauca, Calda, Risaralda y Valle del cauca) para el cultivo de caña**

In [27]:

Departamentos = ['CAUCA', 'CALDAS', 'RISARALDA', 'VALLE DEL CAUCA']
Municipios = ['YUMBO', 'PALMIRA', 'PRADERA', 'EL CERRITO', 'CANDELARIA', 'ROLDANILLO', 'LA UNIÓN', 'TORO',
              'CARTAGO', 'ZARZAL', 'GUADALAJARA DE BUGA', 'BUGALAGRANDE', 'TULUÁ', 'RIOFRÍO', 'FLORIDA', 'CALI', 'JAMUNDÍ',
              'LA VICTORIA', 'OBANDO', 'DAGUA', 'ANDALUCÍA', 'ALCALÁ', 'TRUJILLO', 'BOLÍVAR', 'GUACARÍ',
              'GINEBRA', 'VIJES', 'SAN PEDRO', 'ANSERMANUEVO', 'YOTOCO', 'LA VIRGINIA', 'BALBOA', 'VITERBO',
              'BELALCÁZAR', 'SAN JOSÉ', 'PUERTO TEJADA', 'SANTANDER DE QUILICHAO', 'MIRANDA', 'GUACHENÉ',
              'VILLA RICA', 'CORINTO', 'PADILLA', 'CALOTO', 'BUENOS AIRES']
Cultivo = ['Caña Azucarera', 'Caña', 'Caña Panelera']
# Primer Filtro Departamento
PDatosFiltroD = PDatos[PDatos['departamento'].isin(Departamentos)]
# Segundo filtro por municipio
PDatosFiltroDM = PDatosFiltroD[PDatosFiltroD['municipio'].isin(Municipios)]
# Tercer filtro por cultivo

PDatosFiltroDMC = PDatosFiltroDM[PDatosFiltroDM['cultivo'].isin(Cultivo)]

**K-means**

In [28]:
# Función para determinar de forma gráfica el número óptimo de grupos por el método de codo

def metodo_codo(DataEscalada):
  """
   Esta función implementa el método de codo para determinar de forma visual el número de
   grupos en que debe ser agrupada la submuestra DataEscalada. La función entrega la curva de inercia SSE.
   interactiva donde se puede apreciar las distancias a los centroides (promedios) en función de los grupos
   Entrada: Dataframe.pandas: Dataframe con los datos de las propiedades química del suelo
            seleccionadas normalizadas, se utiliza el z-scores como medida de estandarización
  """
  K = range(1, 11)
  inertias = []

  for k in K:
      km = KMeans(n_clusters=k, random_state=42, n_init='auto')
      km.fit(DataEscalada)
      inertias.append(km.inertia_)

  # Se realiza un gráfico de linea interactivo con plotly
  fig = go.Figure()
  fig.add_trace(go.Scatter(
      x=list(K),
      y=inertias,
      mode='lines+markers'
  ))

  fig.update_layout(
      title="Método del Codo para determinar número óptimo de clusters",
      xaxis_title="Número de clusters (k)",
      yaxis_title="Inercia (SSE)",
      template="plotly_white",
      title_x = 0.5
  )
  fig.show()
  return


# Función principal del programa
# Se inicia eliminado las columna que no tiene información

Columnas_Filtro_Kmeans = ['acidez intercambiable', 'aluminio intercambiable',
                         'manganeso disponible olsen', 'hierro disponible doble acido',
                         'cobre disponible doble acido', 'manganeso disponible doble acido',
                         'zinc disponible doble acido', 'zinc disponible olsen']

Columnas_Modelo_Kmeans = ['ph agua suelo', 'materia organica', 'fosforo bray ii',
                          'azufre fosfato monocalcico', 'calcio intercambiable',
                          'magnesio intercambiable', 'potasio intercambiable',
                          'sodio intercambiable', 'capacidad de intercambio cationico',
                          'conductividad electrica', 'hierro disponible olsen',
                          'cobre disponible', 'boro disponible']

# Se elimina las columnas que no tiene datos o que presenta bastantes datos ausentes
DataKmeans = PDatosFiltroDMC.drop(columns=Columnas_Filtro_Kmeans)
DataKmeans = DataKmeans[Columnas_Modelo_Kmeans]
# Se realiza una imputación utilizando una medida de tendencia central en este caso la mediana
# Se selecciona la mediana dado que es robusta frente a datos extremos presente en los datos
DataKmeans[Columnas_Modelo_Kmeans] = DataKmeans[Columnas_Modelo_Kmeans].fillna(DataKmeans[Columnas_Modelo_Kmeans].median())
# Inicia el agrupamiento
scaler = StandardScaler()
DataKmeanScaler = scaler.fit_transform(DataKmeans)

# Se determina el número de grupos óptimo
metodo_codo(DataKmeanScaler)
# Se selecciona 3 grupos
kmeans = KMeans(n_clusters=3, random_state=42)
# Se realiza teniendo en cuenta los tres grupos
PDatosFiltroDMC.loc[:, 'grupo'] = kmeans.fit_predict(DataKmeanScaler)
PDatosFiltroDMC

/tmp/ipython-input-916311986.py:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,secuencial,fecha de analisis,departamento,municipio,cultivo,estado,tiempo de establecimiento,topografia,drenaje,riego,...,cobre disponible,manganeso disponible olsen,zinc disponible olsen,boro disponible,hierro disponible doble acido,cobre disponible doble acido,manganeso disponible doble acido,zinc disponible doble acido,año,grupo
381,382,2014-08-01,VALLE DEL CAUCA,TULUÁ,Caña Panelera,Establecido,De 5 a 10 años,Plano,Buen drenaje,Gravedad,...,3.30,1.10,0.800,0.215,NaN,NaN,NaN,NaN,2014,1
4621,4621,2014-10-31,VALLE DEL CAUCA,CALI,Caña Panelera,Por establecer,No indica,Plano,Regular drenaje,Gravedad,...,5.50,2.80,0.600,0.360,NaN,NaN,NaN,NaN,2014,1
7851,7852,2014-02-28,VALLE DEL CAUCA,FLORIDA,Caña Panelera,Establecido,Mas de 10 años,Plano,Buen drenaje,Gravedad,...,3.80,2.90,0.899,0.117,NaN,NaN,NaN,NaN,2014,1
7857,7858,2014-02-28,VALLE DEL CAUCA,FLORIDA,Caña Panelera,Establecido,Mas de 10 años,Plano,Buen drenaje,Gravedad,...,3.90,4.80,0.899,0.117,NaN,NaN,NaN,NaN,2014,1
7858,7859,2014-02-28,VALLE DEL CAUCA,FLORIDA,Caña Panelera,Establecido,Mas de 10 años,Plano,Buen drenaje,Gravedad,...,4.30,4.00,1.600,0.207,NaN,NaN,NaN,NaN,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84214,84214,2023-02-02,VALLE DEL CAUCA,BUGALAGRANDE,Caña Azucarera,ESTABLECIDO,De 0 a 1 año,Plano,Buen drenaje,No Indica,...,2.74,NaN,NaN,0.120,NaN,NaN,NaN,NaN,2023,1
84659,84659,2023-05-19,CAUCA,SANTANDER DE QUILICHAO,Caña Panelera,Por establecer,De 0 a 1 año,Pendiente moderada,Buen drenaje,No Indica,...,2.64,1.76,NaN,0.200,NaN,NaN,NaN,NaN,2023,1
87431,87432,2024-11-13,VALLE DEL CAUCA,PALMIRA,Caña,ESTABLECIDO,De 0 a 1 año,No indica,Buen drenaje,Gravedad,...,6.15,2.19,1.260,0.310,NaN,NaN,NaN,NaN,2024,1
87432,87433,2024-11-13,VALLE DEL CAUCA,PALMIRA,Caña Azucarera,ESTABLECIDO,De 0 a 1 año,No indica,Buen drenaje,Gravedad,...,5.74,1.44,1.010,0.250,NaN,NaN,NaN,NaN,2024,2


**Se extraen los datos, cada dos años y por grupo**

**Al final se genera un total de** $Periodos \times nclusters$

**Donde** $ nclusters → \text{Número de cluster óptimo ver método del codo}$

In [29]:
# Se generan los intervalos de análisis
Intervalos = [2014, 2018, 2022, 2026]  # 3 Períodos de análisis
PDatosFiltroDMC = PDatosFiltroDMC.copy()
PDatosFiltroDMC.loc[:, 'intervalos'] = pd.cut(PDatosFiltroDMC['año'], bins=Intervalos, right=False)
PDatosFiltroDMC['intervalos'] = PDatosFiltroDMC['intervalos'].astype('category')
# Se generan los grupos por año y categoría (0 - 1 - 2 - 3)
GruposDatos = PDatosFiltroDMC.groupby(['grupo', 'intervalos'], observed=False)
#Grupo1 = GruposDatos.get_group((0, pd.Interval(2014, 2018, closed='left')))
GruposDatos.size().reset_index().rename(columns={0: 'Tamaño'})

,grupo,intervalos,Tamaño
0,0,"[2014, 2018)",27
1,0,"[2018, 2022)",13
2,0,"[2022, 2026)",0
3,1,"[2014, 2018)",344
4,1,"[2018, 2022)",125
5,1,"[2022, 2026)",26
6,2,"[2014, 2018)",89
7,2,"[2018, 2022)",93
8,2,"[2022, 2026)",15


**Inicia el proceso de detección de los datos anómalos o atípicos, se implementa 3 métodos distintos combinados entre si, la selección del método dependerá del tamaño del grupo (ver resultados anteriores columna Tamaño). A continuación se comparte un algortimo que ilustra el método adoptado**

**Siglas:**

$z: \text{hace referencia a z-scores robusto}$

$iso: \text{hace referencia insolation forest} + z$

$iqr: \text{hace referencia a rango intercuartílico}$

**Algortimo**

$Si \text{ Tamaño} \geq 25$
  
$ \quad resultado \leftarrow iso(subgrupos, columnas, zthresh)$

$\text{De lo contrario si } 10 \leq \text{ Tamaño} < 25$

$\quad resultados ← z(subgrupos, columnas, zthresh)$

$\text{De lo contrario si } 0 < \text{ Tamaño} < 9$

$\quad resultados ← iqr(subgrupos, columnas)$

$\text{De lo contrario si }$

$\quad Imprimir('Grupo sin datos')$

**Hiperparámetros de los algoritmos de detección asumidos en esta implementación**

$zthresh ← 3.5$

$ϵ = 1 \times 10^{-9}$

$IQR_{Factor} = 1.5$

**Salidas**

Excel denominado Resultados_Finales.xlsx donde en la columna 'metodo anomalia' etiqueta si el dato presenta anomalia, los resultados pueden ser:

$\text{metodo anomalia} → iso$ anomalía detectada por el método iso

$\text{metodo anomalia} → z$ anomalía detectada por el método z

$\text{metodo anomalia} → iqr$ anomalía detectada por el método iqr

$\text{metodo anomalia} → ok$ sin anomalia detectada dato confiable

$\text{Resultado_ICD} →$ Dataframe con los valores del ICD calculados, en este trabajo el $ICD$ siguió el siguiente criterio

$ICD = 1- \frac{X_{anomalos} + X_{Nulos}}{X_{total}}$

Donde

$X_{anomalos} : $ Es el número de filas detectadas como anómalas.

$X_{Nulos} : $ Es el número de filas detectadas con datos nulos.

$X_{total} : $ es el número de filas totales (Tamaño) en cada subgrupo.

$ICD → 1 :$ Sin anomalías (caso ideal)

$ICD → 0: $ Caso extremo (todos los datos son anómalos o faltantes)



In [30]:
def _mad(x):
  """ Función de calcula el MAD (desviación absoluta mediana)
  Entrada: x: numpy arreglo.
  Salida: mad: numpy arreglo
  """
  med = np.median(x)
  mad = np.median(np.abs(x - med))
  return mad


def robust_z_scores(arr, axis=0, eps=1e-9):
  """ Función que determina los Z-scores robustos usando la mediana y la desviación absoluta mediana (MAD).
  Método de cálculo
        z= (x - mediana) / (1.4826 * MAD)
  mediana : es la mediana de x
  MAD = mediana(|x - mediana|) medida robusta de la dispersión
  arr: numpy array 2D o 1D
  Devuelve array del mismo shape con z-scores robustos.
  """
  arr = np.asarray(arr)
  if arr.ndim == 1:
      med = np.nanmedian(arr)
      mad = np.nanmedian(np.abs(arr - med))
      denom = max(mad * 1.4826, eps)
      return (arr - med) / denom
  else:
      med = np.nanmedian(arr, axis=axis)
      # broadcast med para restar
      diff = np.abs(arr - np.expand_dims(med, axis=axis))
      mad = np.nanmedian(diff, axis=axis)
      denom = np.maximum(mad * 1.4826, eps)
      # evitar división por cero
      return (arr - np.expand_dims(med, axis=axis)) / np.expand_dims(denom, axis=axis)


def isolation_forest_robust_zscore(df_sub, columns,
                                   z_thresh=3.5,
                                   iso_contamination=0.03,
                                   iso_random_state=42,
                                   min_rows_for_iso=25):
  """
  Combina detección univariante robusta (Z-score basado en MAD) con un detector multivariante (IsolationForest) para marcar filas anómalas.

  Generalidades:
      - Calcula Z-scores robustos por columna (mediana / MAD) y, para cada fila, conserva el valor máximo absoluto de z (robust_z_max).
      - Marca z_flag = True si robust_z_max > z_thresh.
      - Si el número de filas en df_sub >= min_rows_for_iso, ejecuta
            IsolationForest sobre las columnas especificadas:
                - Imputa NaNs por la mediana de la columna antes de ajustar/predict.
                - iso_score <- salida de `decision_function` (mayor => menos anomalía).
                - iso_flag <- True para predicciones == -1 (anomalía).
                - combined_flag <- z_flag OR iso_flag.
        - Añade columnas de diagnóstico al dataframe de salida.

  Entradas:

        - df_sub : pandas.DataFrame DataFrame de entrada (no se modifica en sitio; la función devuelve una copia).
        - columns : lista o tupla. Lista con los nombres de las columnas numéricas que se usarán para los cálculos.
        - z_thresh : float, umbral del z.score robusto por defecto 3.5
        - iso_contamination : float, default=0.03. Proporción esperada de anomalías para IsolationForest (parámetro contamination).
        - iso_random_state : int, default=42. Semilla para reproducción en IsolationForest.
        - min_rows_for_iso : int, default=25. Número mínimo de filas requerido para ejecutar IsolationForest. Si hay menos, no se ajusta el modelo multivariante.
     -
  Salida
    Tipo pandas.DataFrame
        Copia de df_sub con las siguientes columnas añadidas:
        - robust_z_max (float): máximo del |z_robust| por fila.
        - z_flag (bool): True si robust_z_max > z_thresh.
        - iso_score (float): valor de decision_function de IsolationForest (cuanto mayor, menos anomalía). NaN si no se ejecutó.
        - iso_flag (bool): True si IsolationForest predice anomalía (-1).
        - combined_flag (bool): z_flag OR iso_flag.
        - n_rows_group (int): número de filas del subgrupo.
        - metodo_anomalia (str): etiqueta corta ('iso' si combined_flag True, '' o 'ok' si no).

    Notas
    -IsolationForest **no** admite NaNs; la función imputa NaNs por la mediana de cada columna antes de ajustar y predecir. La imputación es local a la copia interna.
    - iso_score sigue la convención de sklearn: valores más bajos indican mayor anomalía.
    - Para conjuntos pequeños (menor que min_rows_for_iso) se omite IsolationForest; en ese caso iso_score será NaN y iso_flag False.
    - Ajusta iso_contamination: valores demasiado altos marcan muchas filas como anomalías; valores muy bajos pueden dejar pasar outliers.
  """
  df = df_sub.copy()
  X = df[columns].astype(float)  # asegurar floats
  # --- Robust Z-scores univariantes ---
  # calcular z por columna
  z_arr = np.zeros_like(X.values, dtype=float)
  for j, col in enumerate(columns):
      col_vals = X.iloc[:, j].values
      z_col = robust_z_scores(col_vals)
      z_arr[:, j] = z_col
  # max abs across columns
  robust_z_max = np.nanmax(np.abs(z_arr), axis=1)
  df['robust_z_max'] = robust_z_max
  df['z_flag'] = df['robust_z_max'] > z_thresh

  # --- Isolation Forest (multivariante) ---
  n_rows = df.shape[0]
  df['iso_score'] = np.nan
  df['iso_flag'] = False

  if n_rows >= min_rows_for_iso:
      # decide contamination adaptativo si quieres
      iso = IsolationForest(contamination=iso_contamination,
                            random_state=iso_random_state)
      # manejar NaNs: IsolationForest no admite NaNs -> imputar con mediana por columna
      X_imp = X.copy()
      for col in columns:
          if X_imp[col].isna().any():
              med = X_imp[col].median()
              X_imp[col] = X_imp[col].fillna(med)
      iso.fit(X_imp.values)
      scores = iso.decision_function(X_imp.values)  # cuanto mayor => menos anomalía
      preds = iso.predict(X_imp.values)             # -1 anomalía, 1 normal
      df['iso_score'] = scores
      df['iso_flag'] = preds == -1
  else:
      # Opcional: para n pequeño, podrías ajustar iso_contamination y correr igualmente
      df['iso_score'] = np.nan
      df['iso_flag'] = False

  # --- Combinar flags (union por defecto) ---
  df['combined_flag'] = df['z_flag'] | df['iso_flag']

  # Añadir columnas de diagnóstico opcional
  df['n_rows_group'] = n_rows
  df.loc[:, 'metodo anomalia'] = df['combined_flag'].apply(lambda b: 'iso' if b else '')
  return df


def robust_zscore_only(df, cols, z_thresh=3.5):
  """
  Función que determina los Z-scores robustos usando la mediana y la desviación absoluta mediana (MAD).
  Método de cálculo
        z= (x - mediana) / (1.4826 * MAD)
  mediana : es la mediana de x
  MAD = mediana(|x - mediana|) medida robusta de la dispersión
  arr: numpy array 2D o 1D
  Devuelve array del mismo shape con z-scores robustos.
  Entradas:
          - tipo Dataframe.pandas: df es el dataframe del subgrupo
          - tipo lista o tupla: lista que contiene las columnas de la variables químicas del suelo a aplicar z-score robusto
          - tipos float opcional: z_thresh es el umbral de decisión valores de z por encima o por debajo de este umbral son
                                  clasificador como atípicos o anómalas
  Salidas:
          - tipo Dataframe.pandas: mismo dataframe de entrada pero con columnas adicionales donde se precisas la transformación realizada.
          Un valor en la columna combined_flag -> True indica valor anómalo (algún valor de las variables químicas del suelo se encuentra por fuera del rango)
          y False (todas las variables dentro del rango ) alguna variable
  """
  df = df.copy()
  if df.shape[0] == 0:
      return df
  eps = 1e-9
  arr = df[cols].to_numpy(dtype=float)  # NxM
  med = np.nanmedian(arr, axis=0)
  # MAD por columna
  diff = np.abs(arr - med)
  mad = np.nanmedian(diff, axis=0)
  scale = np.maximum(mad * 1.4826, eps)  # evita divisiones por cero
  # Broadcasting para producir z-scores
  z = (arr - med) / scale
  z_df = pd.DataFrame(z, index=df.index, columns=[f"z_{c}" for c in cols])
  medians = pd.Series(med, index=cols)
  scales = pd.Series(scale, index=cols)
  for c in z_df.columns:
      df.loc[:, c] = z_df[c]
  df.loc[:, 'combined_flag'] = (z_df.abs() > z_thresh).any(axis=1)
  # no hay iso
  df.loc[:, 'metodo anomalia'] = df['combined_flag'].apply(lambda b: 'z' if b else '')
  return df


def iqr(df_sub, cols, factor=1.5):
  """
   Función que determinar datos atípicos por medio del filtro de rango intercuartílico
   Método
        Umbral Superior = Q3 + 1.5(Q3 - Q1)
        Umbral Inferior = Q1 - 1.5(Q3 - Q1)
   Entradas
         - Tipo Dataframe.pandas. df_sub: DataFrame a filtrar.
        - Tipo lista o tupla. cols columnas aplicar el filtro
   Salida:
        tipo Dataframe.pandas: mismo dataframe de entrada con columnas adicionales.
        - Un valor en la columna combined_flag -> True indica valor anómalo (algún valor de las variables químicas del suelo se encuentra por fuera del rango)
          y False (todas las variables dentro del rango ) alguna variable
  """
  df = df_sub.copy()

  # Calcular Q1 y Q3 para todas las columnas a la vez
  Q1 = df[cols].quantile(0.25)
  Q3 = df[cols].quantile(0.75)
  IQR = Q3 - Q1

  # Límites
  lower = Q1 - factor * IQR
  upper = Q3 + factor * IQR

  # Comparación vectorizada
  outliers = (df[cols] < lower) | (df[cols] > upper)

  # Si alguna columna es outlier, marca la fila completa
  df['combined_flag'] = outliers.any(axis=1)
  df['metodo anomalia'] = df['combined_flag'].apply(lambda b: 'iqr' if b else '')
  return df


def ICD(df, tamano, columns, anomaly_col='combined_flag'):
  """
   Función que determina el índice de calidad de los datos (ICD)
   Método:
              ICD = 1- (X_anomalos + X_nulos) /(Tamaño_Grupo)
   Entrada:
     -Tipo Dataframe.pandas. df Dataframe con la culumna de combined_flag
     - tipo int o float. tamano variable que contiene el tamaño del grupo
     - tipo lista o tupla: columna donde se verifica si hay datos nulo o no identificados
    Salida:
       - tipo float: ICD
  """
  if tamano == 0:
    return
  Anomalias = df[anomaly_col].sum()
  Nulos = df[columns].isna().any(axis=1).sum()
  return 1 - ((Anomalias + Nulos) / (tamano))


# Función principal
tamanos = []
metodo = []
lista_resultados = []
numero_anomalias = 0
indices_calidad = {}
# Caso de estudio se prueba la metodología con un total de 8 grupos
for (grupo, intervalo), subgrupos in GruposDatos:
  Tamano = subgrupos.shape[0]
  if Tamano >= 25:
    resultado = isolation_forest_robust_zscore(subgrupos, Columnas_Modelo_Kmeans,
                                               z_thresh=3.5,
                                               iso_contamination=0.03,
                                               min_rows_for_iso=13)
    resultado1 = resultado.copy()
    numero_anomalias = resultado['combined_flag'].sum() + numero_anomalias
    print("Anomalías detectadas:", numero_anomalias)
    metodo.append('ISO')
  elif 10 <= Tamano < 25:
    resultado = robust_zscore_only(subgrupos, Columnas_Modelo_Kmeans,
                       z_thresh=3.5)
    numero_anomalias = resultado['combined_flag'].sum() + numero_anomalias
    metodo.append('ZSR')
    resultado2 = resultado.copy()
  elif 0 < Tamano < 10:
    resultado = iqr(subgrupos, Columnas_Modelo_Kmeans)
    numero_anomalias = resultado['combined_flag'].sum() + numero_anomalias
    print("Anomalías detectadas:", numero_anomalias)
    metodo.append('IQR')
  elif Tamano == 0:
    print('Sin datos')
  tamanos.append(Tamano)
  indices_calidad[(grupo, intervalo)] = ICD(resultado, Tamano, Columnas_Modelo_Kmeans)
  lista_resultados.append(resultado)
Resultado_ICD = pd.DataFrame(
    [(grupo, intervalo, dqi) for (grupo, intervalo), dqi in indices_calidad.items()],
    columns=['Grupo', 'Intervalo', 'ICD']
    )
Resultado_ICD['Tamaño'] = tamanos
Resultado_ICD['Metodo'] = metodo

Data_final_anomalias = pd.concat(lista_resultados, axis=0, join='inner', ignore_index=True)
Data_final_anomalias['metodo anomalia'] = Data_final_anomalias['metodo anomalia'].replace({'': 'ok'})
Data_final_anomalias.to_excel('Resultados_Finales.xlsx')


Anomalías detectadas: 8
Anomalías detectadas: 135
Anomalías detectadas: 180
Anomalías detectadas: 193
Anomalías detectadas: 220
Anomalías detectadas: 242


In [31]:
Resultado_ICD

,Grupo,Intervalo,ICD,Tamaño,Metodo
0,0,"[2014, 2018)",0.703704,27,ISO
1,0,"[2018, 2022)",0.461538,13,ZSR
2,1,"[2014, 2018)",0.415698,344,ISO
3,1,"[2018, 2022)",0.128000,125,ISO
4,1,"[2022, 2026)",0.000000,26,ISO
5,2,"[2014, 2018)",0.662921,89,ISO
6,2,"[2018, 2022)",0.473118,93,ISO
7,2,"[2022, 2026)",0.400000,15,ZSR


**Gráficos interactivo que resume los resultados obtenidos**

In [32]:
Intervalos = ['[2014, 2018)', '[2018, 2022)', '[2022, 2026)']
Resultado_ICD['Intervalo'] = Resultado_ICD['Intervalo'].astype(str)
Resultado_ICD['Intervalo'] = pd.Categorical(Resultado_ICD['Intervalo'],
                                             categories=Intervalos,
                                             ordered=True)
subtitulos = ['Diagrama de dispersión Método vs ICD', 'Diagrama de dispersión Intervalos vs ICD',
              'Diagrama de dispersión Támaño vs ICD', 'Mapa de Calor Intervalos vs ICD']
# Se va a gráficar con plotly un subplot de 2x2
heat = Resultado_ICD.pivot(index='Grupo', columns='Intervalo', values='ICD').sort_index()
Figura = make_subplots(rows=2, cols=2,
                       subplot_titles=subtitulos,
                       )

for annot in Figura['layout']['annotations']:
    annot['font'] = dict(size=20)

# Diagarma de dispersión Método vs ICD
for grupo, dfg in Resultado_ICD.groupby('Grupo'):
    Figura.add_trace(
        go.Scatter(
            x=dfg['Metodo'],
            y=dfg['ICD'],
            mode='markers',
            marker=dict(size=12, symbol='circle'),
            hovertemplate=f'Grupo={grupo}<br>Método=%{{x}}<br>ICD=%{{y}}<br>Tamaño=%{{customdata[0]}}<br>Intervalo=%{{customdata[1]}}',
            customdata=dfg[['Tamaño', 'Intervalo']]
        ),
        row=1, col=1
    )

# Diagrama de dispersión Período de tiempo vs ICD
for grp, dfg in Resultado_ICD.groupby('Grupo'):
    sizes = dfg['ICD'] * 30 + 8 # ejemplo: tamaño proporcional al ICD
    Figura.add_trace(
        go.Scatter(
            x=dfg['Intervalo'].astype(str),
            y=dfg['ICD'],
            mode='lines+markers',
            name=f"Grupo {grp}",
            marker=dict(size=sizes),
            hovertemplate=f'Grupo={grp}<br>Período=%{{x}}<br>ICD=%{{y:.2f}}<extra></extra><br>Método=%{{customdata}}',
            customdata=dfg['Metodo']
        ),
        row=1, col=2
    )

# Diagrama de dispersión Tamaño del grupo vs ICD
for metodo, dfg in Resultado_ICD.groupby('Metodo'):
    Figura.add_trace(
        go.Scatter(
            x=dfg['Tamaño'],
            y=dfg['ICD'],
            mode='markers',
            name=metodo.strip(),
            marker=dict(size=12, symbol='circle'),
            hovertemplate=f'Método={metodo}<br>Tamaño=%{{x}}<br>ICD=%{{y}}<br>Grupo=%{{customdata[0]}}<br>Intervalo=%{{customdata[1]}}',
            customdata=dfg[['Grupo', 'Intervalo']]
        ),
        row=2, col=1
    )

# Heatmap mapa de calor del ICD por grupo y período de análisis
Figura.add_trace(go.Heatmap(z=heat.values, x=heat.columns.astype(str), y=heat.index.astype(str),
                            colorscale='YlOrRd', zmin=0, zmax=1,
                            colorbar=dict(title='ICD',
                                          x=1.05,  # mover la colorbar a la derecha del subplot
                                          y=0.23,
                                          len=0.5
                                          ),
                             hovertemplate='Período=%{x}<br>Grupo=%{y}<br>ICD=%{z:.2f}<extra></extra>'
                            ),
                    row=2, col=2)

# Configuración del estilo del gráfico interactivo

Figura.update_xaxes(title='Intervalo de análisis', row=2, col=2,
                    title_font=dict(size=18))

Figura.update_xaxes(title='Método', row=1, col=1,
                    title_font=dict(size=18))

Figura.update_xaxes(title='Tamaño', row=2, col=1,
                    title_font=dict(size=18))

Figura.update_yaxes(title='Grupo', row=2, col=2,
                    title_font=dict(size=15))

Figura.update_layout(height=800, showlegend=True, title_x=0.5,
                     title_text='Dashboard ICD — Resumen',
                     margin=dict(l=50, r=150, t=100, b=50),
                     barmode='group', title_font=dict(size=20))

Figura.add_annotation(text='ICD',
                      xref='paper',
                      yref='paper',
                      x=-0.035,
                      y=0.5,
                      showarrow=False,
                      textangle=-90,
                      font=dict(size=18)
                      )

Figura.show()